# 명탐정 코난 매니아 판별기 (LangGraph)

## OpenAI LLM 준비 및 퀴즈 파일 지정
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`
* 핵심 실습 환경이라 `gpt-4o` 사용

In [ ]:
import gradio as gr
import random
import json

from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END

from langchain_teddynote.graphs import visualize_graph

# 초기 설정 및 데이터 로딩
QUIZ_FILE = "data/quizzes.json"
QUIZ_COUNT = 3
QUIZ_COMMANDS = ["퀴즈", "퀴즈 시작"]

load_dotenv()
llm = ChatOpenAI(model="gpt-4o", temperature=0.5)


## 퀴즈 문항 (`conan_quiz.json`) 로딩
- `def load_quizzes()` :	load_quizzes라는 이름의 함수를 정의
- `with open(QUIZ_FILE, "r", encoding="utf-8") as f`: 지정된 경로의 JSON 파일(QUIZ_FILE)을 UTF-8 인코딩 방식으로 읽기 모드("r")로 열고, with 구문을 사용하여 파일 사용 후 자동으로 닫히도록 처리
- `all_q = json.load(f)` : JSON 파일 내용을 파싱하여 파이썬 객체(리스트 형태, 퀴즈 문제들)로 변환.
- `return random.sample(all_q, QUIZ_COUNT)` :전체 문제 중 QUIZ_COUNT 개수만큼 무작위로 샘플링하여 반환(중복 없이 선택)

In [ ]:
# 퀴즈 로딩 함수
def load_quizzes():
    with open(QUIZ_FILE, "r", encoding="utf-8") as f:
        all_q = json.load(f)
    return random.sample(all_q, QUIZ_COUNT)

## 데이터 구조 정의
### 기본 데이터 정의

`Pydantic`의 `BaseModel`을 상속한 데이터 모델 클래스 정의
- `GradingResult(BaseModel)` : 개별 문제 하나에 대한 채점 결과
    - `Field(description="...")`: description은 LLM에게 이 필드가 무엇을 의미하는지 알려주는 중요한 힌트
- `FinalReport(BaseModel)` : 모든 문제의 채점 결과와 최종 점수를 포함하는 최종 보고서
    - `results: list[GradingResult]`: GradingResult의 리스트(list) 형태
- `llm_with_final_report = llm.with_structured_output(FinalReport)` : LLM의 출력을 FinalReport 클래스의 JSON 형식으로 지정

In [ ]:
class GradingResult(BaseModel):
    """단일 문제에 대한 채점 결과를 상세히 담는 클래스입니다."""
    
    question: str = Field(description="채점 대상 문제")
    correct_answer: str = Field(description="문제의 정답")
    user_answer: str = Field(description="사용자가 제출한 답변")
    is_correct: bool = Field(description="정답 여부")
    explanation: str = Field(description="정답에 대한 친절한 해설")

class FinalReport(BaseModel):
    """퀴즈의 모든 채점 결과와 최종 점수를 종합한 최종 보고서 클래스입니다."""

    results: list[GradingResult] = Field(description="각 문제별 채점 결과 리스트")
    total_score: str = Field(description="'총점: X/Y' 형식의 최종 점수 요약")

# LLM 출력을 FinalReport 형식으로 지정
llm_with_final_report = llm.with_structured_output(FinalReport)

### 리스트 리듀서 함수 정의
  - `LangGraph`의 `add_messages` 리듀서는 BaseMessage 객체로 자동 변환 기능을 제공 --> UI 대화 목록으로 사용하기 어려움.
  - `reduce_list`: 단순히 두개의 리스트를 합치는 리듀서 함수 정의 

In [ ]:
# add_messages를 사용하지 않고 단순히 리스트를 합치는 함수
def reduce_list(left: list, right: list) -> list:
    """두 리스트를 합칩니다."""

    return left + right


### 상태 정의

그래프 노드에서 처리되는 상태 클래스로 `AppState(TypedDict)`는 그래프(graph)가 관리할 상태 정보 (dict 형식)
  - `chat_history`: Gradio UI 채팅창에 출력되는 사용자 입력과 퀴즈 문항 저장소로 사용자와 LLM이 주고 받는 메시지와 구분됨.
  - `reduce_list`: UI 대화 목록(`chat_history`) 과 사용자 답변(`user_answers`)은 입력창을 통해 누적되는 목록.

※ LangGraph의 모든 노드 함수들은 상태 클래스를 인자로 받고 처리된 상태 필드들을 포함한 dict를 리턴값으로 전달.

In [ ]:

# 모든 필드를 선택적(total=False)으로 관리 => 상태 관리에 적합한 방식
class AppState(TypedDict, total=False):
    """
    퀴즈 애플리케이션의 전체 상태.
    Annotated를 사용하여 각 필드의 설명을 타입 시스템의 일부로 관리합니다.
    """

    # --- 워크플로우 전반에 사용되는 필드 ---
    user_input: Annotated[str, "사용자의 현재 입력 텍스트"]
    chat_history: Annotated[list[tuple], "UI 채팅창에 표시될 대화 기록", reduce_list]

    # --- 퀴즈 진행 관련 필드 ---
    questions: Annotated[list[dict], "생성된 퀴즈 질문 목록"]
    user_answers: Annotated[list[str], "사용자가 제출한 답변 목록", reduce_list]
    quiz_index: Annotated[int, "현재 진행 중인 quiz의 인덱스 (0부터 시작)"]

    # --- 채점 및 보고서 생성 관련 필드 ---
    grading_prompt: Annotated[str, "채점을 위해 LLM에 전달할 종합 프롬프트"]
    final_report: Annotated[FinalReport, "최종 채점 결과 보고서 객체"]    

## LangGraph 노드 함수 구성
LangGraph 워크플로우에서는 각 단계(노드)가 상태(state)를 받아서, 필요한 상태를 변경한 뒤 반환하는 방식으로 동작
- 입력: state라는 TypedDict 형식의 퀴즈 전체 상태(`AppState`)를 받음
- 출력: 내부 값들이 갱신된(변경된) 동일 타입의 state 객체를 반환 → LangGraph 노드는 항상 **상태(state) → 상태(state)** 의 형태를 유지


### 퀴즈 시작(상태 초기화) 노드
* 상태 갱신의 목적: "퀴즈 시작"이라는 트리거에서 호출되고 모든 상태 정보를 초기화
    - 문제 리스트 세팅: `state["questions"]`
    - 진행 인덱스 초기화: `state["quiz_index"]`
    - 답변 내역, 리포트 등 모두 초기화
    - 유저(`user`)/AI(`assistant`) 대화 이력(`chat_history`)에 안내 메시지 추가

* `questions = load_quizzes()` : 랜덤하게 생성된 QUIZ_COUNT로 개수의 퀴즈 문항

In [ ]:
def quiz_setter(state: AppState) -> AppState:
    """퀴즈를 시작하고 상태를 초기화합니다."""
    
    questions = load_quizzes()
    if not questions:
        return {
            "chat_history": [
                ("assistant", "퀴즈를 불러오는 데 실패했거나 풀 수 있는 문제가 없습니다.")
            ],
            "questions": [],
        }

    return {
        "questions": questions,
        "quiz_index": 0,
        "user_answers": [],
        "final_report": None, 
        "chat_history": [("assistant", "명탐정 코난 퀴즈를 시작합니다!")],
    }


### 문제 출제 노드
* 상태(state) 갱신의 목적 : 문제를 사용자에게 출력(`chat_history`에 포맷팅된 문제 내용을 assistant 메시지로 추가)
    - `quiz_index`에 해당하는 문제(`quiz`)를 가져와 포맷(문자열 변환)
    - 문제 타입이 `multiple_choice`면, 선택지도 보기 형태로 문자열로 추가
    - `chat_history`에 ("assistant", 문제 텍스트) 튜플을 append

In [ ]:
def quiz_popper(state: AppState) -> AppState:
    """현재 quiz_index에 맞는 문제를 포맷하여 chat_history에 추가합니다."""

    quiz_index = state["quiz_index"]
    quiz = state["questions"][quiz_index]

    text = f"문제 {quiz_index + 1}: {quiz['question']}"
    if quiz["type"] == "multiple_choice":
        choices = [f"{i + 1}. {c}" for i, c in enumerate(quiz["choices"])]
        text += "\n" + "\n".join(choices)
   
    return {
        "chat_history": [("assistant", text)],
    }

### 사용자 답변 처리 노드
* 상태(state) 갱신의 목적 : 사용자의 답변(`user_input`)을 처리
    - 현재 문제(`quiz_index`)에 맞는 답변을 파싱, 전처리, 정제
    - 선택형 문제(`multiple_choice`)라면 사용자가 번호로 입력했을 때 실제 보기 텍스트로 변환
    - 가공/정제된 답변(`processed_answer`)을 `user_answers` 리스트에 append
    - 진행 인덱스(`quiz_index`)를 +1 하여 다음 문제로 이동할 준비

* `user_input = state["user_input"].strip()` : `state["user_input"]`은 `chat_fn` 에서 입력됨. `strip()`은 앞뒤 공백 모두 제거
* `except (ValueError, IndexError): pass` : 사용자가 "a"처럼 숫자로 변환할 수 없는 값을 입력하거나(ValueError) "5"처럼 선택지 범위를 벗어나는 숫자를 입력하면(IndexError), pass 즉 아무런 작업도 하지 않고 그냥 넘어가서, 사용자가 입력한 값이 `state["user_answers"]`로 저장됨.

In [ ]:
def answer_collector(state: AppState) -> AppState:
    """사용자 답변을 처리하고 저장한 뒤, 다음 문제로 넘어갑니다."""
    
    quiz_index = state["quiz_index"]
    quiz = state["questions"][quiz_index]
    user_input = state["user_input"].strip()

    # 빈 입력일 경우 안내 메시지만 추가하고, 인덱스는 그대로 유지
    if not user_input:
        return {
            "chat_history": [("assistant", "답변을 입력해 주세요.")],
        }

    processed_answer = user_input
    if quiz["type"] == "multiple_choice":
        try:
            sel = int(user_input) - 1
            if 0 <= sel < len(quiz["choices"]):
                processed_answer = quiz["choices"][sel]
        except (ValueError, IndexError):
            pass

    return {
        "user_answers": [processed_answer],
        "quiz_index": quiz_index + 1
    }

### 채점을 위해 LLM에 전달할 프롬프트 생성 노드
* 상태(state) 갱신의 목적 : 
    - 사용자에게 "채점 중" 안내를 위해 `chat_history`에 추가
    - 채점 대상 데이터를 `state["grading_prompt"]`에 저장 → 다음 노드(LLM 채점 호출 등)에서 활용
* `zip(questions, user_answers)` : 각 문제와 각 답변을 쌍으로 묶어서 반환 (튜플: (문제, 답변))
* `enumerate(...)` : 튜플 쌍에 인덱스 번호(`i`)를 붙여서 (`i, (q, a)`) 형태로 반환

In [ ]:
def grading_prompter(state: AppState) -> AppState:
    """채점을 위해 LLM에 전달할 프롬프트를 생성합니다."""
    
    questions = state["questions"]
    user_answers = state["user_answers"]

    prompt_buff = [
        "지금부터 아래의 문제와 정답, 그리고 사용자의 답변을 보고 채점을 시작해주세요."
    ]
    for i, (q, a) in enumerate(zip(questions, user_answers)):
        prompt_buff.append(f"\n--- 문제 {i + 1} ---")
        prompt_buff.append(f"문제: {q['question']}")
        if q["type"] == "multiple_choice":
            prompt_buff.append(f"선택지: {', '.join(q['choices'])}")
        prompt_buff.append(f"정답: {q['answer']}")
        prompt_buff.append(f"사용자 답변: {a}")

    return {
        "chat_history": [("assistant", "채점을 진행합니다...")],
        "grading_prompt": "\n".join(prompt_buff),
    }


### LLM 채점 및 파싱 노드
* 상태(state) 갱신의 목적 : 
    - 채점 대상 데이터를 `state["grading_prompt"]`를 LLM에 전달
    - LLM의 채점 결과를 FinalReport 클래스 JSON 데이터로 수신하여 `state["final_report"]`에 저장
    
- `chain = prompt | llm_with_final_report` : 프롬프트와 FinalReport를 출력으로 하는 LLM을 LCEL 체인으로 연결



In [ ]:
def grade_reporter(state: AppState) -> AppState:
    """LLM을 호출하여 채점하고 결과를 파싱합니다."""
    
    system_message = """
    당신은 '명탐정 코난' 퀴즈의 전문 채점관입니다. 주어진 문제, 정답, 사용자 답변을 바탕으로 채점해주세요. 
    각 문제에 대해 정답 여부를 판단하고 친절한 해설을 덧붙여주세요. 
    모든 채점이 끝나면, 마지막에는 '총점: X/Y' 형식으로 최종 점수를 반드시 요약해서 보여줘야 합니다. 
    반드시 지정된 JSON 형식으로만 답변해야 합니다."""

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            ("human", "{grading_data}"),
        ]
    )

    try:
        # 체인 생성
        chain = prompt | llm_with_final_report
        report = chain.invoke({"grading_data": state["grading_prompt"]})
        return {
            "final_report": report
        }
    except Exception as e:
        print(f"채점 중 오류 발생: {e}")
        error_report = FinalReport(results=[], total_score="채점 오류가 발생했습니다.")
        return {
            "final_report": error_report
        }


### 최종 리포트(FinalReport)를 사용자에게 보여줄 문자열로 변환 노드
상태(state) 갱신의 목적 : LLM이 제공한 최종 채점 결과를 문제별로 정오, 정답, 제출 답변, 해설을 보기 좋게 포매팅
- 마지막에 총점도 포함
- 안내 메시지(재시작 방법 등)도 추가
- 결과를 `assistant` 메시지로 `chat_history`에 append

워크플로우 마지막 단계로 구조화된 채점 결과 객체(`final_report`)를 사람이 읽을 수 있는 문자열로 변환하여 `chat_history`에 추가함으로써 사용자에게 "마지막 답변"을 제공

In [ ]:
def report_formatter(state: AppState) -> AppState:
    """파싱된 최종 리포트를 사용자에게 보여줄 문자열로 변환합니다."""
    
    final_report = state["final_report"]
    report_buff = ["채점이 완료되었습니다! 🎉\n"]

    if final_report and final_report.results:
        for i, res in enumerate(final_report.results):
            is_correct_text = "✅ 정답" if res.is_correct else "❌ 오답"
            report_buff.append(f"--- 문제 {i + 1} ---")
            report_buff.append(f"문제: {res.question}")
            report_buff.append(f"정답: {res.correct_answer}")
            report_buff.append(f"제출한 답변: {res.user_answer}")
            report_buff.append(f"결과: {is_correct_text}")
            report_buff.append(f"해설: {res.explanation}\n")
        report_buff.append(f"**{final_report.total_score}**")
    else:
        report_buff.append("채점 결과를 생성하는 데 실패했습니다.")

    report_buff.append("\n퀴즈를 다시 시작하려면 '퀴즈 시작'이라고 입력해주세요.")    
    return {
        "chat_history": [("assistant", "\n".join(report_buff))],
    }

### 퀴즈 시작 명령어 예외 처리 노드
사용자가 “퀴즈 시작” 명령어를 입력하지 않은 경우, 올바른 명령어 사용법(트리거 문구)을 안내하는 메시지를 `chat_history`에 `assistant` 역할로 추가

In [ ]:
def input_helper(state: AppState) -> AppState:
    """퀴즈 시작 명령어가 아닐 경우 안내 메시지를 추가합니다."""

    help_message = f"'{" 또는 ".join(QUIZ_COMMANDS)}'이라고 입력하면 퀴즈가 시작됩니다."    
    return {
        "chat_history": [("assistant", help_message)],
    }

### StateGraph 조건부 함수
분기(`Edge Condition`) 함수로, 입력(state)을 분석해서 다음 실행할 노드 이름을 반환
- `continue_quiz_condition` : 퀴즈가 아직 남아 있는지, 아니면 채점 단계로 넘어가야 하는지 다음 노드 이름을 결
- `input_router` : 사용자의 첫 입력이 이미 퀴즈를 진행 중인 상태인지, “퀴즈 시작” 명령어인지, 또는 잘못된 입력인지 구분하여 시작 노드를 결정

In [ ]:
def continue_quiz_condition(state: AppState) -> str:
    """퀴즈를 계속할지, 채점을 시작할지 결정합니다."""

    if state["quiz_index"] < len(state["questions"]):
        return "continue_quiz"
    else:
        return "grade_quiz"


def input_router(state: AppState) -> str:
    """사용자의 입력을 분석하여 워크플로우의 시작점을 결정합니다."""
    
    if state.get("questions") and state["questions"]:
        return "answer_collector"
    else:
        if state["user_input"].strip().lower() in QUIZ_COMMANDS:
            return "quiz_setter"
        else:
            return "input_helper"

## StateGraph 정의 및 컴파일

### 노드 추가
- 각 노드는 하나의 “기능적 단계”를 담당 : `quiz_setter`는 상태 초기화, `quiz_popper`은 문제 출제, `answer_collector`는 답변 처리 등
- `input_helper`는 잘못된 시작 입력 안내

### 조건부 진입점 설정
- 워크플로우의 초기 입력을 분석하여 “퀴즈 시작” 명령이면 `quiz_setter`, 이미 진행 중이면 `answer_collector`, 그 외엔 `input_helper`로 각기 다른 노드에서 실행이 시작됨

### 노드 연결(엣지 설정)

직접 연결:
- `quiz_setter` → `quiz_popper`: 퀴즈 시작 후 문제 출제
- `grading_prompter` → `grade_reporter` → `report_formatter`: 채점 준비→실행→리포트 출력
- `report_formatter` → `END`: 채점 결과 출력 후 종료

조건부 연결:
- `answer_collector` 다음에 `continue_quiz_condition`로 검사, 문제 남아 있으면 `continue_quiz`, 다 풀었으면 `grade_quiz`
- 노드 종료 연결 : `quiz_popper` → `END`, `input_helper` → `END` (`quiz_popper → END`는 “한 문제만 내는” 임시 흐름처럼 보이니, 여러 문제 반복엔 반드시 위 조건부 연결이 필요!)

In [ ]:
graph = StateGraph(AppState)

# 노드 추가
graph.add_node("quiz_setter", quiz_setter)
graph.add_node("quiz_popper", quiz_popper)
graph.add_node("answer_collector", answer_collector)
graph.add_node("grading_prompter", grading_prompter)
graph.add_node("grade_reporter", grade_reporter)
graph.add_node("report_formatter", report_formatter)
graph.add_node("input_helper", input_helper)

# 조건부 진입점 설정
graph.set_conditional_entry_point(
    input_router,
    {
        "quiz_setter": "quiz_setter",
        "answer_collector": "answer_collector",
        "input_helper": "input_helper",
    },
)

# 엣지 연결
graph.add_edge("quiz_setter", "quiz_popper")
graph.add_edge("quiz_popper", END)
graph.add_edge("input_helper", END)

graph.add_conditional_edges(
    "answer_collector",
    continue_quiz_condition,
    {"continue_quiz": "quiz_popper", "grade_quiz": "grading_prompter"},
)
graph.add_edge("grading_prompter", "grade_reporter")
graph.add_edge("grade_reporter", "report_formatter")
graph.add_edge("report_formatter", END)

# 그래프 컴파일 : 퀴즈 애플리케이션 워크플로우 완성
quiz_app = graph.compile()

## 그래프 시각화

In [ ]:
visualize_graph(quiz_app)

## UI 인터페이스 함수
`init_state()` :
- 앱을 초기화할 때 사용할 빈 상태(딕셔너리) 반환
- `app_state` 아래에 문제/대화 이력만 먼저 준비
- 다른 필드는 워크플로우 도중 필요에 따라 추가됨(예: quiz_index, user_answers, final_report 등)

`chat_fn(user_input, state)` : Gradio 챗봇 UI에서 실제로 호출되는 메인 처리 함수


In [ ]:
def init_state():
    return {"app_state": {"questions": [], "chat_history": []}}


def chat_fn(user_input, state):
    app_state = state["app_state"]

    if app_state.get("final_report") and user_input.strip().lower() in QUIZ_COMMANDS:
        app_state = init_state()["app_state"]

    current_chat_history = app_state.get("chat_history", [])
    current_chat_history.append(("user", user_input))

    # 그래프 입력 생성 : **app_state (딕셔너리 언패킹)
    graph_input = {
        **app_state,
        "user_input": user_input,
        "chat_history": current_chat_history,
    }

    # 그래프 실행 : 리턴값은 그래프 노드 함수에서 처리된 AppState 형식의 dict로 변환
    new_state = quiz_app.invoke(graph_input)
    state["app_state"] = new_state

    chat_display = [
        {"role": role, "content": content}
        for role, content in new_state["chat_history"]
    ]

    return chat_display, state

## Gradio UI 정의
Gradio의 Blocks UI 구성 방식을 사용해 명탐정 코난 퀴즈 챗봇 인터페이스를 구성

In [ ]:
# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("### 🕵️ 명탐정 코난 매니아 판별기")

    chatbot = gr.Chatbot(
        label="명탐정 코난 퀴즈 챗봇",
        height=400,
        avatar_images=("data/avatar_user.png", "data/avatar_conan.png"),
        type="messages",
    )

    txt = gr.Textbox(placeholder="'퀴즈 시작'을 입력해보세요!", show_label=False)
    state = gr.State(init_state())

    # 이벤트 핸들링
    txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])

    # 텍스트 박스 초기화
    txt.submit(lambda: "", None, txt)

    demo.launch()

In [ ]:
demo.close()

-----
** End of Documents **